In [2]:
import os
import sys

print(os.getcwd())
cwd0 = './config/'
sys.path.append(cwd0)

import visualID_Eng as vID
from visualID_Eng import  fg, hl, bg
vID.init(cwd0)
import tools4PytChem as t4p

/home/romuald/ENSEIGNEMENT/0-JupyterNotebooks/PytChem


**Start at:** Friday 06 October 2023, 15:58:56  
**Hostname:** insa-12842 (Linux)

<p style="text-align: center"><img width="800px" src="./config/svg/PytChemBanner.svg" style="margin-left:auto; margin-right:auto"/></p>

# 3D NPs. Generation of coordinates

## Utils

In [7]:
import numpy as np
from ase.io import write
from ase import Atoms
from ase.visualize import view

In [8]:
def RAB(coord,a,b):
    import numpy as np
    """calculate the interatomic distance between two atoms a and b"""
    r = np.sqrt((coord[a][0]-coord[b][0])**2 + (coord[a][1]-coord[b][1])**2 + (coord[a][2]-coord[b][2])**2)
    return r

def Rbetween2Points(p1,p2):
    import numpy as np
    r = np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)
    return r

def vector(coord,a,b):
    import numpy as np
    v = [coord[b][0]-coord[a][0],coord[b][1]-coord[a][1],coord[b][2]-coord[a][2]]
    v = np.array(v)
    return v

def vectorBetween2Points(p1,p2):
    import numpy as np
    v = [p2[0]-p1[0],p2[1]-p1[1],p2[2]-p1[2]]
    v = np.array(v)
    return v

def coord2xyz(coord):
    x = np.array(coord)[:,0]
    y = np.array(coord)[:,1]
    z = np.array(coord)[:,2]
    return x,y,z

def vertex(x, y, z, scale):
    """ Return vertex coordinates fixed to the unit sphere """
    length = np.sqrt(x**2 + y**2 + z**2)
    return [(i * scale) / length for i in (x,y,z)]

    
def RadiusSphereAfterV(V):
    return (3*V/(4*np.pi))**(1/3)

def MakeFaceCoord(Rnn,f,coord,nAtomsOnFaces,coordFaceAt):
    # adapt these two lines for square faces or other
    # the idea here is to interpolate between edge atoms of two relevant edges
    # (for example two opposite edges of a squared face)
    # be careful of the vectors orientation of the edges!
    if (len(f) == 3):  #triangular facet
        edge1 = [f[1],f[0]]
        edge2 = [f[1],f[2]]
    if (len(f) == 5):  #pentagonal facet
        edge1 = [f[1],f[0]]
        edge2 = [f[1],f[2]]
    nAtomsOnEdges = int((RAB(coord,f[1],f[0])+1e-6)/Rnn) - 1
    nIntervalsE = nAtomsOnEdges + 1
    #print("shell = ",i,"nAtomsOnEdges = ",nAtomsOnEdges)
    for n in range(nAtomsOnEdges):
        CoordAtomOnEdge1 = coord[edge1[0]]+vector(coord,edge1[0],edge1[1])*(n+1) / nIntervalsE
        CoordAtomOnEdge2 = coord[edge2[0]]+vector(coord,edge2[0],edge2[1])*(n+1) / nIntervalsE
        #print(CoordAtomOnEdge1)
        #print(CoordAtomOnEdge2)
        distBetween2EdgeAtoms = Rbetween2Points(CoordAtomOnEdge1,CoordAtomOnEdge2)
        #print("R = ",distBetween2EdgeAtoms)
        nAtomsBetweenEdges = int((distBetween2EdgeAtoms+1e-6)/Rnn) - 1
        nIntervalsF = nAtomsBetweenEdges + 1
        for m in range(nAtomsBetweenEdges):
            coordFaceAt.append(CoordAtomOnEdge1 + vectorBetween2Points(CoordAtomOnEdge1,CoordAtomOnEdge2)*(m+1) / nIntervalsF)
            nAtomsOnFaces += 1
    return nAtomsOnFaces,coordFaceAt

def centerOfGravity(c,select):
    nselect = len(select)
    xg = 0
    yg = 0
    zg = 0
    for at in select:
        xg += c[at][0]
        yg += c[at][1]
        zg += c[at][2]
    cog = [xg/nselect, yg/nselect, zg/nselect]
    return cog

# def principal_moment_of_inertia(mol):
#     from ase import Atoms
#     com = mol.get_center_of_mass()
#     m = mol.get_masses()
#     nAtoms = len(mol)
#     x,y,z = coord2xyz(mol.)

## Icosahedra

### Regular icosahedra

In [9]:
from ase.cluster import Icosahedron

atoms = Icosahedron("Au", 8, 4.1536)
atoms.center(vacuum=10)

write("3DNP-coords/ico.xyz", atoms)
write("3DNP-coords/ico.cif", atoms)

In [10]:
class regIC:
    import numpy as np
    nFaces = 20
    nEdges = 30
    nVertices = 12
    phi = (1 + np.sqrt(5))/2 # golden ratio
    edgeLengthF = 1
    radiusCSF = np.sqrt(10 + 2*np.sqrt(5))/4
    interShellF = 1/radiusCSF
#    interShellF = np.sqrt(2*(1-1/np.sqrt(5)))
    radiusISF = np.sqrt(3) * (3 + np.sqrt(5))/12
  
    def __init__(self, Rnn, nShell):
        self.Rnn = Rnn
        self.nShell = nShell
        self.nAtoms = 0
        self.nAtomsPerShell = [0]
        self.interShellDistance = self.Rnn / self.interShellF
          
    def __str__(self):
        return(f"Regular icosahedron with {self.nShell} shell(s) and Rnn = {self.Rnn}")
    
    def nAtomsF(self,i):
        return (10*i**3 + 11*i)//3 + 5*i**2 + 1
    
    def nAtomsPerShellAnalytic(self):
        n = []
        Sum = 0
        for i in range(self.nShell+1):
            Sum = sum(n)
            ni = self.nAtomsF(i)
            n.append(ni-Sum)
        return n
    
    def nAtomsAnalytic(self):
        n = self.nAtomsF(self.nShell)
        return n
    
    def edgeLength(self):
        return self.Rnn*self.nShell

    def radiusCircumscribedSphere(self):
        return self.radiusCSF*self.edgeLength()

    def radiusInscribedSphere(self):
        return self.radiusISF*self.edgeLength()

    def area(self):
        el = self.edgeLength()
        return 5 * el**2 * np.sqrt(3)
    
    def volume(self):
        el = self.edgeLength()
        return 5 * el**3 * (3 + np.sqrt(5))/12
    
    def MakeVertices(self,i):
        import sys
        """
        input:
            - i = index of the shell
        returns:
            - CoordVertices = the 12 vertex coordinates of the ith shell of an icosahedron
            - edges = indexes of the 30 edges
            - faces = indexes of the 20 faces 
        """
        if (i == 0):
            CoordVertices = [0., 0., 0.]
            edges = []
            faces = []
        elif (i > self.nShell):
            sys.exit(f"icoreg.MakeVertices(i) is called with i = {i} > nShell = {self.nShell}")
        else:
            phi = self.phi
            scale = self.interShellDistance * i
            CoordVertices = [ vertex(-1, phi, 0, scale),\
                       vertex( 1, phi, 0, scale),\
                       vertex(-1, -phi, 0, scale),\
                       vertex( 1, -phi, 0, scale),\
                       vertex(0, -1, phi, scale),\
                       vertex(0, 1, phi, scale),\
                       vertex(0, -1, -phi, scale),\
                       vertex(0, 1, -phi, scale),\
                       vertex( phi, 0, -1, scale),\
                       vertex( phi, 0, 1, scale),\
                       vertex(-phi, 0, -1, scale),\
                       vertex(-phi, 0, 1, scale) ]
            edges = [( 1, 0), ( 3, 2), ( 4, 2), ( 4, 3), ( 5, 0), ( 5, 1), ( 5, 4), ( 6, 2), ( 6, 3), ( 7, 0),\
                  ( 7, 1), ( 7, 6), ( 8, 1), ( 8, 3), ( 8, 6), ( 8, 7), ( 9, 1), ( 9, 3), ( 9, 4), ( 9, 5),\
                  ( 9, 8), (10, 0), (10, 2), (10, 6), (10, 7), (11, 0), (11, 2), (11, 4), (11, 5), (11,10),]
            faces = [(7,0,1),(7,1,8),(7,8,6),(7,6,10),(7,10,0),\
                     (0,11,5),(0,5,1),(1,5,9),(1,8,9),(8,9,3),(8,3,6),(6,3,2),(6,10,2),(10,2,11),(10,0,11),\
                     (4,2,3),(4,3,9),(4,9,5),(4,5,11),(4,11,2)]
            edges = np.array(edges)
            CoordVertices = np.array(CoordVertices)
            faces = np.array(faces)
        return CoordVertices, edges, faces

    def coords(self):
        # central atom = "1st shell"
        c = [[0., 0., 0.]]
        self.nAtoms = 1
        self.nAtomsPerShell = [0]
        self.nAtomsPerShell[0] = 1
        indexVertexAtoms = []
        indexEdgeAtoms = []
        indexFaceAtoms = []
        for i in range(1,self.nShell+1):
            # vertices
            nAtoms0 = self.nAtoms
            cshell, E, F = self.MakeVertices(i)
            self.nAtoms += self.nVertices
            self.nAtomsPerShell.append(self.nVertices)
            c.extend(cshell.tolist())
            indexVertexAtoms.extend(range(nAtoms0,self.nAtoms))

            # intermediate atoms on edges e
            nAtoms0 = self.nAtoms
            Rvv = RAB(cshell,E[0,0],E[0,1]) #distance between two vertex atoms
            nAtomsOnEdges = int((Rvv+1e-6) / self.Rnn)-1
            nIntervals = nAtomsOnEdges + 1
            #print("nAtomsOnEdges = ",nAtomsOnEdges)
            coordEdgeAt = []
            for n in range(nAtomsOnEdges):
                for e in E:
                    a = e[0]
                    b = e[1]
                    coordEdgeAt.append(cshell[a]+vector(cshell,a,b)*(n+1) / nIntervals)
            self.nAtomsPerShell[i] += nAtomsOnEdges * len(E) # number of edges x nAtomsOnEdges
            self.nAtoms += nAtomsOnEdges * len(E)
            c.extend(coordEdgeAt)
            indexEdgeAtoms.extend(range(nAtoms0,self.nAtoms))
            
            # now, facet atoms
            coordFaceAt = []
            nAtomsOnFaces = 0
            nAtoms0 = self.nAtoms
            for f in F:
                nAtomsOnFaces,coordFaceAt = MakeFaceCoord(self.Rnn,f,cshell,nAtomsOnFaces,coordFaceAt)
            self.nAtomsPerShell[i] += nAtomsOnFaces
            self.nAtoms += nAtomsOnFaces
            c.extend(coordFaceAt)
            indexFaceAtoms.extend(range(nAtoms0,self.nAtoms))
            
        # print(indexVertexAtoms)
        # print(indexEdgeAtoms)
        # print(indexFaceAtoms)
        return c,[indexVertexAtoms,indexEdgeAtoms,indexFaceAtoms]
    
    def prop(self):
        print(self)
        print("number of vertices = ",self.nVertices)
        print("number of edges = ",self.nEdges)
        print("number of faces = ",self.nFaces)
        print("phi = ",self.phi)
        print(f"intershell factor = {self.interShellF:.2f}")
        print(f"nearest neighbour distance = {self.Rnn:.2f} Å")
        print(f"intershell distance = {self.interShellDistance:.2f} Å")
        print(f"edge length = {self.edgeLength()*0.1:.2f} nm")
        print(f"radius after volume = {RadiusSphereAfterV(self.volume()*1e-3):.2f} nm")
        print(f"radius of the circumscribed sphere = {self.radiusCircumscribedSphere()*0.1:.2f} nm")
        print(f"radius of the inscribed sphere = {self.radiusInscribedSphere()*0.1:.2f} nm")
        print(f"area = {self.area()*1e-2:.1f} nm2")
        print(f"volume = {self.volume()*1e-3:.1f} nm3")
        print("number of atoms per shell = ",self.nAtomsPerShellAnalytic())
        print("total number of atoms = ",self.nAtomsAnalytic())

In [11]:
IC1 = regIC(2.7,4)
IC1.prop()
c1,i1 = IC1.coords()
print(IC1.nAtoms)
print(IC1.nAtomsPerShell)
ico1 = Atoms('Au'*IC1.nAtoms, positions=c1)
write("3DNP-coords/ico1.xyz", ico1)
ico1.moi = ico1.get_moments_of_inertia() # in amu*angstrom**2
print(f"Moments of inertia = {ico1.moi[0]:.2f} {ico1.moi[1]:.2f} {ico1.moi[2]:.2f} amu.Å2")
ico1.masses = ico1.get_masses()
ico1.M = ico1.masses.sum()
ico1.moiM = ico1.moi/ico1.M
print(f"Moments of inertia / M = {ico1.moiM[0]:.2f} {ico1.moiM[1]:.2f} {ico1.moiM[2]:.2f} amu.Å2")
ico1.dim = 2*np.sqrt(5*ico1.moiM)
print(f"Size of the ellipsoid = {ico1.dim[0]*0.1:.2f} {ico1.dim[1]*0.1:.2f} {ico1.dim[2]*0.1:.2f} nm")

#view(ico1)
write("3DNP-coords/ico1.xyz", ico1)
print()
IC2 = regIC(2.7,10)
IC2.prop()
c2,i2 = IC2.coords()
print(IC2.nAtoms)
print(IC2.nAtomsPerShell)
ico2 = Atoms('Au'*IC2.nAtoms, positions=c2)
#view(ico2)
write("3DNP-coords/ico2.xyz", ico2)
ico2.moi = ico2.get_moments_of_inertia() # in amu*angstrom**2
ico2.masses = ico2.get_masses()
ico2.M = ico2.masses.sum()
ico2.moiM = ico2.moi/ico2.M
print(f"Moments of inertia / M = {ico2.moiM[0]:.2f} {ico2.moiM[1]:.2f} {ico2.moiM[2]:.2f} amu.Å2")
ico2.dim = 2*np.sqrt(5*ico2.moiM)
print(f"Size of the ellipsoid = {ico2.dim[0]*0.1:.2f} {ico2.dim[1]*0.1:.2f} {ico2.dim[2]*0.1:.2f} nm")

print()
IC3 = regIC(2.7,20)
IC3.prop()
c3,i3 = IC3.coords()
print(IC3.nAtoms)
print(IC3.nAtomsPerShell)
ico3 = Atoms('Au'*IC3.nAtoms, positions=c3)
#view(ico2)
write("3DNP-coords/ico3.xyz", ico3)

Regular icosahedron with 4 shell(s) and Rnn = 2.7
number of vertices =  12
number of edges =  30
number of faces =  20
phi =  1.618033988749895
intershell factor = 1.05
nearest neighbour distance = 2.70 Å
intershell distance = 2.57 Å
edge length = 1.08 nm
radius after volume = 0.87 nm
radius of the circumscribed sphere = 1.03 nm
radius of the inscribed sphere = 0.82 nm
area = 10.1 nm2
volume = 2.7 nm3
number of atoms per shell =  [1, 12, 42, 92, 162]
total number of atoms =  309
309
[1, 12, 42, 92, 162]
Moments of inertia = 2341672.64 2341672.64 2341672.64 amu.Å2
Moments of inertia / M = 38.47 38.47 38.47 amu.Å2
Size of the ellipsoid = 2.77 2.77 2.77 nm

Regular icosahedron with 10 shell(s) and Rnn = 2.7
number of vertices =  12
number of edges =  30
number of faces =  20
phi =  1.618033988749895
intershell factor = 1.05
nearest neighbour distance = 2.70 Å
intershell distance = 2.57 Å
edge length = 2.70 nm
radius after volume = 2.17 nm
radius of the circumscribed sphere = 2.57 nm
radiu

## Tetrahedra

### Regular tetrahedron

In [12]:
class regTd:
    import numpy as np
    nFaces = 4
    nEdges = 6
    nVertices = 4
    edgeLengthF = 2
    heightOfPyramid = edgeLengthF * np.sqrt(2/3)
    radiusCSF = edgeLengthF * np.sqrt(3/8)
    radiusISF = edgeLengthF/np.sqrt(24)
    radiusMSF = edgeLengthF/np.sqrt(8)
    fveAngle = np.rad2deg(np.arccos(1/np.sqrt(3)))
    fefAngle = np.rad2deg(np.arccos(1/3))
    vcvAngle = np.rad2deg(np.arccos(-1/3))
  
    def __init__(self, Rnn, nSlice):
        self.Rnn = Rnn
        self.nSlice = nSlice
        self.nAtoms = 0
        self.nAtomsPerSlice = []
        self.interSliceDistance = self.Rnn / self.heightOfPyramid
          
    def __str__(self):
        return(f"Regular dodecahedron with {self.nSlice} slice(s) and Rnn = {self.Rnn}")
    
    def nAtomsF(self,i):
        return i**3/6 + i**2 + 11*i/6 + 1
    
    def nAtomsPerSliceAnalytic(self):
        n = []
        Sum = 0
        for i in range(self.nSlice+1):
            Sum = sum(n)
            ni = self.nAtomsF(i)
            n.append(ni-Sum)
            print(i,ni,Sum,n)
        return n
    
    def nAtomsAnalytic(self):
        n = self.nAtomsF(self.nSlice)
        return n
    
    def edgeLength(self):
        return self.Rnn*self.nSlice

    def radiusCircumscribedSphere(self):
        return self.radiusCSF*self.edgeLength()

    def radiusInscribedSphere(self):
        return self.radiusISF*self.edgeLength()

    def radiusMidSphere(self):
        return self.radiusMSF*self.edgeLength()

    def area(self):
        el = self.edgeLength()
        return el**2*np.sqrt(3)
    
    def volume(self):
        el = self.edgeLength()
        return el**3/(6*np.sqrt(2)) 

    def MakeVertices(self,i):
        import sys
        """
        input:
            - i = index of the slice
        returns:
            - CoordVertices = the 4 vertex coordinates of the ith slice of a tetrahedron
            - edges = indexes of the 6 edges
            - faces = indexes of the 4 faces 
        """
        if (i > self.nSlice):
            sys.exit(f"icoreg.MakeVertices(i) is called with i = {i} > nSlice = {self.nSlice}")
        else:
            scale = self.interSliceDistance * i
            CoordVertices = [vertex(1, 1, 1, scale),\
                             vertex(1, -1, -1, scale),\
                             vertex(-1, 1, -1, scale),\
                             vertex(-1, -1, 1, scale)]
            edges = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]
            faces = [(0,2,1),(0,1,3),(0,3,2),(1,2,3)]
            edges = np.array(edges)
            CoordVertices = np.array(CoordVertices)
            faces = np.array(faces)
        return CoordVertices, edges, faces

    def coords(self):
        c = []
        print(self.nAtomsPerSlice)
        indexVertexAtoms = []
        indexEdgeAtoms = []
        indexFaceAtoms = []

        # vertices
        nAtoms0 = 0
        self.nAtoms += self.nVertices
        cSlice, E, F = self.MakeVertices(self.nSlice)
        c.extend(cSlice.tolist())
        indexVertexAtoms.extend(range(nAtoms0,self.nAtoms))

        # intermediate atoms on edges e
        nAtoms0 = self.nAtoms
        Rvv = RAB(cSlice,E[0,0],E[0,1]) #distance between two vertex atoms
        nAtomsOnEdges = int((Rvv+1e-6) / self.Rnn)-1
        nIntervals = nAtomsOnEdges + 1
        #print("nAtomsOnEdges = ",nAtomsOnEdges)
        coordEdgeAt = []
        for n in range(nAtomsOnEdges):
            for e in E:
                a = e[0]
                b = e[1]
                coordEdgeAt.append(cSlice[a]+vector(cSlice,a,b)*(n+1) / nIntervals)
        self.nAtoms += nAtomsOnEdges * len(E)
        c.extend(coordEdgeAt)
        indexEdgeAtoms.extend(range(nAtoms0,self.nAtoms))
        
        # now, facet atoms
        coordFaceAt = []
        nAtomsOnFaces = 0
        nAtoms0 = self.nAtoms
        for f in F:
            nAtomsOnFaces,coordFaceAt = MakeFaceCoord(self.Rnn,f,c,nAtomsOnFaces,coordFaceAt)
        self.nAtoms += nAtomsOnFaces
        c.extend(coordFaceAt)
        indexFaceAtoms.extend(range(nAtoms0,self.nAtoms))

#         for i in range(0,):
#             print(i,"Slice")
#             # vertices
#             self.nAtoms += self.nVertices
#             self.nAtomsPerSlice.append(self.nVertices)
#             c.extend(cSlice.tolist())
#             indexVertexAtoms.extend(range(nAtoms0,self.nAtoms))

#             # intermediate atoms on edges e
#             nAtoms0 = self.nAtoms
#             Rvv = RAB(cSlice,E[0,0],E[0,1]) #distance between two vertex atoms
#             nAtomsOnEdges = int((Rvv+1e-6) / self.Rnn)-1
#             nIntervals = nAtomsOnEdges + 1
#             #print("nAtomsOnEdges = ",nAtomsOnEdges)
#             coordEdgeAt = []
#             for n in range(nAtomsOnEdges):
#                 for e in E:
#                     a = e[0]
#                     b = e[1]
#                     coordEdgeAt.append(cSlice[a]+vector(cSlice,a,b)*(n+1) / nIntervals)
#             self.nAtomsPerSlice[i] += nAtomsOnEdges * len(E) # number of edges x nAtomsOnEdges
#             self.nAtoms += nAtomsOnEdges * len(E)
#             c.extend(coordEdgeAt)
#             indexEdgeAtoms.extend(range(nAtoms0,self.nAtoms))
            
        return c,[indexVertexAtoms,indexEdgeAtoms,indexFaceAtoms]
    
    def prop(self):
        print(self)
        print("number of vertices = ",self.nVertices)
        print("number of edges = ",self.nEdges)
        print("number of faces = ",self.nFaces)
        print(f"nearest neighbour distance = {self.Rnn:.2f} Å")
        print(f"edge length = {self.edgeLength()*0.1:.2f} nm")
        print(f"height of pyramid = {self.heightOfPyramid*0.1:.2f} nm")
        print(f"radius after volume = {RadiusSphereAfterV(self.volume()*1e-3):.2f} nm")
        print(f"radius of the circumscribed sphere = {self.radiusCircumscribedSphere()*0.1:.2f} nm")
        print(f"radius of the inscribed sphere = {self.radiusInscribedSphere()*0.1:.2f} nm")
        print(f"radius of the midsphere that is tangent to edges = {self.radiusMidSphere()*0.1:.2f} nm")
        print(f"area = {self.area()*1e-2:.1f} nm2")
        print(f"volume = {self.volume()*1e-3:.1f} nm3")
        print(f"face-vertex-edge angle = {self.fveAngle:.1f}°")
        print(f"face-edge-face (dihedral) angle = {self.fefAngle:.1f}°")
        print(f"vertex-center-vertex (tetrahedral bond) angle = {self.vcvAngle:.1f}°")
        print("number of atoms per slice = ",self.nAtomsPerSliceAnalytic())
        print("total number of atoms = ",self.nAtomsAnalytic())

In [13]:
Td1 = regTd(2.7,6)
Td1.prop()
c1,i1 = Td1.coords()
print(f"Total number of atoms = {Td1.nAtoms:.0f}")
print(f"Number of atoms per slice = {Td1.nAtomsPerSlice}")
Td1 = Atoms('Au'*Td1.nAtoms, positions=c1)
write("3DNP-coords/Td1.xyz", Td1)
Td1.moi = Td1.get_moments_of_inertia() # in amu*angstrom**2
print(f"Moments of inertia = {Td1.moi[0]:.2f} {Td1.moi[1]:.2f} {Td1.moi[2]:.2f} amu.Å2")
Td1.masses = Td1.get_masses()
Td1.M = Td1.masses.sum()
Td1.moiM = Td1.moi/Td1.M
print(f"Moments of inertia / M = {Td1.moiM[0]:.2f} {Td1.moiM[1]:.2f} {Td1.moiM[2]:.2f} amu.Å2")
Td1.dim = 2*np.sqrt(5*Td1.moiM)
print(f"Size of the ellipsoid = {Td1.dim[0]*0.1:.2f} {Td1.dim[1]*0.1:.2f} {Td1.dim[2]*0.1:.2f} nm")

Regular dodecahedron with 6 slice(s) and Rnn = 2.7
number of vertices =  4
number of edges =  6
number of faces =  4
nearest neighbour distance = 2.70 Å
edge length = 1.62 nm
height of pyramid = 0.16 nm
radius after volume = 0.49 nm
radius of the circumscribed sphere = 1.98 nm
radius of the inscribed sphere = 0.66 nm
radius of the midsphere that is tangent to edges = 1.15 nm
area = 4.5 nm2
volume = 0.5 nm3
face-vertex-edge angle = 54.7°
face-edge-face (dihedral) angle = 70.5°
vertex-center-vertex (tetrahedral bond) angle = 109.5°
0 1.0 0 [1.0]
1 4.0 1.0 [1.0, 3.0]
2 10.0 4.0 [1.0, 3.0, 6.0]
3 20.0 10.0 [1.0, 3.0, 6.0, 10.0]
4 35.0 20.0 [1.0, 3.0, 6.0, 10.0, 15.0]
5 55.99999999999999 35.0 [1.0, 3.0, 6.0, 10.0, 15.0, 20.999999999999993]
6 84.0 55.99999999999999 [1.0, 3.0, 6.0, 10.0, 15.0, 20.999999999999993, 28.000000000000007]
number of atoms per slice =  [1.0, 3.0, 6.0, 10.0, 15.0, 20.999999999999993, 28.000000000000007]
total number of atoms =  84.0
[]
Total number of atoms = 74
Numbe

## Dodecahedra

In [14]:
class regDD:
    import numpy as np
    nFaces = 12
    nEdges = 30
    nVertices = 20
    phi = (1 + np.sqrt(5))/2 # golden ratio
    edgeLengthF = 1
    radiusCSF = np.sqrt(3) * (1 + np.sqrt(5))/4
    interShellF = 1/radiusCSF
    radiusISF = np.sqrt((5/2) + (11/10)*np.sqrt(5))/2
  
    def __init__(self, Rnn, nShell):
        self.Rnn = Rnn
        self.nShell = nShell
        self.nAtoms = 0
        self.nAtomsPerShell = [0]
        self.interShellDistance = self.Rnn / self.interShellF
          
    def __str__(self):
        return(f"Regular dodecahedron with {self.nShell} shell(s) and Rnn = {self.Rnn}")
    
    def nAtomsF(self,i):
        return 10*i**3 + 15*i**2 + 7*i + 1
    
    def nAtomsPerShellAnalytic(self):
        n = []
        Sum = 0
        for i in range(self.nShell+1):
            Sum = sum(n)
            ni = self.nAtomsF(i)
            n.append(ni-Sum)
        return n
    
    def nAtomsAnalytic(self):
        n = self.nAtomsF(self.nShell)
        return n
    
    def edgeLength(self):
        return self.Rnn*self.nShell

    def radiusCircumscribedSphere(self):
        return self.radiusCSF*self.edgeLength()

    def radiusInscribedSphere(self):
        return self.radiusISF*self.edgeLength()

    def area(self):
        el = self.edgeLength()
        return 3 * el**2 * np.sqrt(25 + 10*np.sqrt(5))
    
    def volume(self):
        el = self.edgeLength()
        return (15 + 7*np.sqrt(5)) * el**2/4 

    def MakeVertices(self,i):
        import sys
        """
        input:
            - i = index of the shell
        returns:
            - CoordVertices = the 20 vertex coordinates of the ith shell of a dodecahedron
            - edges = indexes of the 30 edges
            - faces = indexes of the 12 faces 
        """
        if (i == 0):
            CoordVertices = [0., 0., 0.]
            edges = []
            faces = []
        elif (i > self.nShell):
            sys.exit(f"icoreg.MakeVertices(i) is called with i = {i} > nShell = {self.nShell}")
        else:
            phi = self.phi
            scale = self.interShellDistance * i
            CoordVertices = [vertex(1, 1, 1, scale),\
                             vertex(-1, 1, 1, scale),\
                             vertex(1, -1, 1, scale),\
                             vertex(1, 1, -1, scale),\
                             vertex(-1, -1, 1, scale),\
                             vertex(-1, 1, -1, scale),\
                             vertex(1, -1, -1, scale),\
                             vertex(-1, -1, -1, scale),\
                             vertex(0, phi, 1/phi, scale),\
                             vertex(0, -phi, 1/phi, scale),\
                             vertex(0, phi, -1/phi, scale),\
                             vertex(0, -phi, -1/phi, scale),\
                             vertex(1/phi, 0, phi, scale),\
                             vertex(-1/phi, 0, phi, scale),\
                             vertex(1/phi, 0, -phi, scale),\
                             vertex(-1/phi, 0, -phi, scale),\
                             vertex(phi, 1/phi, 0, scale),\
                             vertex(phi, -1/phi, 0, scale),\
                             vertex(-phi, 1/phi, 0, scale),\
                             vertex(-phi, -1/phi, 0, scale)]
            edges = [(8,0), (8,1), (9,2), (9,4), (10,3), (10,5), (10,8), (11,6), (11,7),\
                     (11,9), (12,0), (12,2), (13,1), (13,4), (13,12), (14,3), (14,6), (15,5),\
                     (15,7), (15,14), (16,0), (16,3), (17,2), (17,6), (17,16), (18,1), (18,5),\
                     (19,4), (19,7), (19,18)]
            faces = [(0,8,10,3,16),(0,12,13,1,8),(8,1,18,5,10),(10,5,15,14,3),(3,14,6,17,16),(16,17,2,12,0),\
                     (4,9,11,7,19),(4,13,12,2,9),(9,2,17,6,11),(11,6,14,15,7),(7,15,5,18,19),(19,18,1,13,4)]
            edges = np.array(edges)
            CoordVertices = np.array(CoordVertices)
            faces = np.array(faces)
        return CoordVertices, edges, faces

    def coords(self):
        # central atom = "1st shell"
        c = [[0., 0., 0.]]
        self.nAtoms = 1
        self.nAtomsPerShell = [0]
        self.nAtomsPerShell[0] = 1
        indexVertexAtoms = []
        indexEdgeAtoms = []
        indexFaceAtoms = []
        for i in range(1,self.nShell+1):
            # vertices
            nAtoms0 = self.nAtoms
            cshell, E, F = self.MakeVertices(i)
            self.nAtoms += self.nVertices
            self.nAtomsPerShell.append(self.nVertices)
            c.extend(cshell.tolist())
            indexVertexAtoms.extend(range(nAtoms0,self.nAtoms))

            # intermediate atoms on edges e
            nAtoms0 = self.nAtoms
            Rvv = RAB(cshell,E[0,0],E[0,1]) #distance between two vertex atoms
            nAtomsOnEdges = int((Rvv+1e-6) / self.Rnn)-1
            nIntervals = nAtomsOnEdges + 1
            #print("nAtomsOnEdges = ",nAtomsOnEdges)
            coordEdgeAt = []
            for n in range(nAtomsOnEdges):
                for e in E:
                    a = e[0]
                    b = e[1]
                    coordEdgeAt.append(cshell[a]+vector(cshell,a,b)*(n+1) / nIntervals)
            self.nAtomsPerShell[i] += nAtomsOnEdges * len(E) # number of edges x nAtomsOnEdges
            self.nAtoms += nAtomsOnEdges * len(E)
            c.extend(coordEdgeAt)
            indexEdgeAtoms.extend(range(nAtoms0,self.nAtoms))
            #print(c)
            
            # center of each pentagonal facet
            nAtomsOnFaces = 0
            nAtoms0 = self.nAtoms
            coordFaceAt = []
            for f in F:
                nAtomsOnFaces += 1
                coordCenterFace = centerOfGravity(cshell,f)
                #print("coordCenterFace",coordCenterFace)
                self.nAtomsPerShell[i] += 1
                coordFaceAt.append(coordCenterFace)
                # atoms from the center of each pentagonal facet to each of its apex 
                nAtomsOnInternalRadius = i-1
                nIntervals = nAtomsOnInternalRadius+1
                print(f)
                for indexApex,apex in enumerate(f):
                    if (indexApex == len(f)-1):
                        indexApexPlus1 = 0
                    else:
                        indexApexPlus1 = indexApex+1
                    apexPlus1 = f[indexApexPlus1]
                    for n in range(nAtomsOnInternalRadius):
                        nAtomsOnFaces += 1
                        coordAtomOnApex = coordCenterFace+vectorBetween2Points(coordCenterFace,cshell[apex])*(n+1) / nIntervals
                        coordAtomOnApexPlus1 = coordCenterFace+vectorBetween2Points(coordCenterFace,cshell[apexPlus1])*(n+1) / nIntervals
                        coordFaceAt.append(coordAtomOnApex)
                        RbetweenRadialAtoms = Rbetween2Points(coordAtomOnApex,coordAtomOnApexPlus1)
                        nAtomsBetweenRadialAtoms = int((RbetweenRadialAtoms+1e-6) / self.Rnn)-1
                        nIntervalsRadial = nAtomsBetweenRadialAtoms + 1
                        for k in range(nAtomsBetweenRadialAtoms):
                            nAtomsOnFaces += 1
                            coordFaceAt.append(coordAtomOnApex+vectorBetween2Points(coordAtomOnApex,coordAtomOnApexPlus1)*(k+1) / nIntervalsRadial)
            self.nAtoms += nAtomsOnFaces
            c.extend(coordFaceAt)
            indexFaceAtoms.extend(range(nAtoms0,self.nAtoms))
                
            # now, facet atoms
            # coordFaceAt = []
            # nAtomsOnFaces = 0
            # nAtoms0 = self.nAtoms
            # for f in F:
            #     print("f = ",f)
            #     #nAtomsOnFaces,coordFaceAt = MakeFaceCoord(self.Rnn,f,cshell,nAtomsOnFaces,coordFaceAt)
            # self.nAtomsPerShell[i] += nAtomsOnFaces
            # self.nAtoms += nAtomsOnFaces
            # c.extend(coordFaceAt)
            # indexFaceAtoms.extend(range(nAtoms0,self.nAtoms))
            
        # print(indexVertexAtoms)
        # print(indexEdgeAtoms)
        # print(indexFaceAtoms)
        return c,[indexVertexAtoms,indexEdgeAtoms,indexFaceAtoms]
    
    def prop(self):
        print(self)
        print("number of vertices = ",self.nVertices)
        print("number of edges = ",self.nEdges)
        print("number of faces = ",self.nFaces)
        print("phi = ",self.phi)
        print(f"intershell factor = {self.interShellF:.2f}")
        print(f"nearest neighbour distance = {self.Rnn:.2f} Å")
        print(f"intershell distance = {self.interShellDistance:.2f} Å")
        print(f"edge length = {self.edgeLength()*0.1:.2f} nm")
        print(f"radius after volume = {RadiusSphereAfterV(self.volume()*1e-3):.2f} nm")
        print(f"radius of the circumscribed sphere = {self.radiusCircumscribedSphere()*0.1:.2f} nm")
        print(f"radius of the inscribed sphere = {self.radiusInscribedSphere()*0.1:.2f} nm")
        print(f"area = {self.area()*1e-2:.1f} nm2")
        print(f"volume = {self.volume()*1e-3:.1f} nm3")
        print("number of atoms per shell = ",self.nAtomsPerShellAnalytic())
        print("total number of atoms = ",self.nAtomsAnalytic())

In [15]:
# DD1 = regdodeca(2.7,1)
# c1,i1 = DD1.coords()
# n = 0
# for i in range(1,DD1.nAtoms):#do not take into account first atom
#     for j in range(1,i):
#         if RAB(c1,i,j) < 2.8:
#             n += 1
#             print(f"{n}  ({i-1},{j-1})") #shift => do not take into account inner atom

In [16]:
DD1 = regDD(2.7,5)
DD1.prop()
c1,i1 = DD1.coords()
print(DD1.nAtoms)
print(DD1.nAtomsPerShell)
dd1 = Atoms('Au'*DD1.nAtoms, positions=c1)
write("3DNP-coords/dd1.xyz", dd1)
dd1.moi = dd1.get_moments_of_inertia() # in amu*angstrom**2
print(f"Moments of inertia = {dd1.moi[0]:.2f} {dd1.moi[1]:.2f} {dd1.moi[2]:.2f} amu.Å2")
dd1.masses = dd1.get_masses()
dd1.M = dd1.masses.sum()
dd1.moiM = dd1.moi/ico1.M
print(f"Moments of inertia / M = {dd1.moiM[0]:.2f} {dd1.moiM[1]:.2f} {dd1.moiM[2]:.2f} amu.Å2")
dd1.dim = 2*np.sqrt(5*dd1.moiM)
print(f"Size of the ellipsoid = {dd1.dim[0]*0.1:.2f} {dd1.dim[1]*0.1:.2f} {dd1.dim[2]*0.1:.2f} nm")

Regular dodecahedron with 5 shell(s) and Rnn = 2.7
number of vertices =  20
number of edges =  30
number of faces =  12
phi =  1.618033988749895
intershell factor = 0.71
nearest neighbour distance = 2.70 Å
intershell distance = 3.78 Å
edge length = 1.35 nm
radius after volume = 0.69 nm
radius of the circumscribed sphere = 1.89 nm
radius of the inscribed sphere = 1.50 nm
area = 37.6 nm2
volume = 1.4 nm3
number of atoms per shell =  [1, 32, 122, 272, 482, 752]
total number of atoms =  1661
[ 0  8 10  3 16]
[ 0 12 13  1  8]
[ 8  1 18  5 10]
[10  5 15 14  3]
[ 3 14  6 17 16]
[16 17  2 12  0]
[ 4  9 11  7 19]
[ 4 13 12  2  9]
[ 9  2 17  6 11]
[11  6 14 15  7]
[ 7 15  5 18 19]
[19 18  1 13  4]
[ 0  8 10  3 16]
[ 0 12 13  1  8]
[ 8  1 18  5 10]
[10  5 15 14  3]
[ 3 14  6 17 16]
[16 17  2 12  0]
[ 4  9 11  7 19]
[ 4 13 12  2  9]
[ 9  2 17  6 11]
[11  6 14 15  7]
[ 7 15  5 18 19]
[19 18  1 13  4]
[ 0  8 10  3 16]
[ 0 12 13  1  8]
[ 8  1 18  5 10]
[10  5 15 14  3]
[ 3 14  6 17 16]
[16 17  2 12  

# Geometry optimization

# End

In [17]:
vID.end(cwd0)

**End at:** Friday 06 October 2023, 15:59:34  
**Duration:** 00:00:38 359ms

<p style="text-align: center"><img width="800px" src="./config/svg/logoEnd.svg" style="margin-left:auto; margin-right:auto"/></p>